In [ ]:
from google.colab import drive

drive.mount('./drive')

Mounted at ./drive


In [ ]:
from shutil import copytree
import os
import time
import datetime
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import random
from IPython import display
from tqdm import tqdm
import PIL.Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras import backend as K
from keras.layers import Input, Activation, Add, UpSampling2D, BatchNormalization, Conv2D, Dropout, Conv2DTranspose, Lambda, Flatten, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from tensorflow.keras.utils import plot_model
from IPython.display import Image
from tensorflow.keras.applications.vgg16 import VGG16
from keras.callbacks import TensorBoard

import cv2
import PIL
import json, os
import sys

import glob
import itertools

In [ ]:
copytree('drive/MyDrive/Colab Notebooks/deblurring_module', '/content/deblurring_module')
!mkdir GOPRO_Large

!unzip -qq "/content/drive/MyDrive/Colab Notebooks/GOPRO_Large.zip" -d "/content/GOPRO_Large"

NameError: ignored

In [ ]:
# 데이터 불러오기
from deblurring_module.data_load import reorganize_gopro_files
reorganize_gopro_files('GOPRO_Large/', 'images')

dir: 100%|██████████| 2/2 [00:47<00:00, 23.95s/it]


In [ ]:
from deblurring_module.images import load_images
data = load_images('./images/train', -1)
y_train, x_train = data['B'], data['A']

In [ ]:
x_train = (x_train - 127.5) / 127.5
y_train = (y_train - 127.5) / 127.5

In [ ]:
def res_block(input, filters, kernel_size=(3,3), strides=(1,1), use_dropout=False):

    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if use_dropout:
        x = Dropout(0.5)(x)

    x = Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding='same')(x)
    x = BatchNormalization()(x)

    #  Two convolution layers are connected between input and output 
    merged = Add()([input, x])
    return merged

In [ ]:
image_shape = (256, 256, 3)
ngf = 64
input_nc = 3
batch_size = 16
output_nc = 3

input_shape_generator = (256, 256, input_nc)
n_block_gen = 9

def generator_model():
    """ Build a generation model """
    # Current version : ResNet block
    inputs = Input(shape=(256,256,3))

    x = Conv2D(filters=ngf, kernel_size=(7, 7), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Increase filter number
    n_downsampling = 2
    for i in range(n_downsampling):
        mult = 2 ** i
        x = Conv2D(filters=ngf * mult * 2, kernel_size=(3, 3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # 9 ResNet blocks
    mult = 2 ** n_downsampling
    for i in range(n_block_gen):
        x = res_block(x, ngf * mult, use_dropout=True)

    #  Reduce the convolution kernel to 3 individual  (RGB)
    for i in range(n_downsampling):
        mult = 2 ** (n_downsampling - i)
        x = Conv2DTranspose(filters=int(ngf * mult / 2), kernel_size=(3, 3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    x = Conv2D(filters=output_nc, kernel_size=(7, 7), padding='same')(x)
    x = Activation('tanh')(x)

    # Add direct connection from input to output and recenter to [-1, 1]
    outputs = Add()([x, inputs])
    outputs = Lambda(lambda z: z / 2)(outputs)

    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model

In [ ]:
ndf = 64

def discriminator_model():
    """ Construct discrimination model ."""
    n_layers, use_sigmoid = 3, False
    inputs = Input(shape=(256, 256, 3))

    x = Conv2D(filters=ndf, kernel_size=(4, 4), strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    nf_mult, nf_mult_prev = 1, 1
    for n in range(n_layers):
        nf_mult_prev, nf_mult = nf_mult, min(2 ** n, 8)
        x = Conv2D(filters=ndf * nf_mult, kernel_size=(4, 4), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

    nf_mult_prev, nf_mult = nf_mult, min(2 ** n_layers, 8)
    x = Conv2D(filters=ndf * nf_mult, kernel_size=(4, 4), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters=1, kernel_size=(4, 4), strides=1, padding='same')(x)
    if use_sigmoid:
        x = layers.Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(1024, activation='tanh')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x, name='Discriminator')
    return model

In [ ]:
generator = generator_model()
pretrained_path = '/content/drive/MyDrive/photo_results/725/generator_59_25.01026725769043.h5'
generator.load_weights(pretrained_path)

In [ ]:
discriminator = discriminator_model()
dis_pretrained_path = '/content/drive/MyDrive/photo_results/725/discriminator_59.h5'
discriminator.load_weights(dis_pretrained_path)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [ ]:
generator.summary()

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
def train(train, label, epochs):
    for epoch in tqdm(range(epochs)):
        start = time.time()
        permutated_indexes = np.random.permutation(train.shape[0])

        len_index = int(train.shape[0]/batch_size)
        
        for index in range(len_index):
            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
                image_blur_batch = train[batch_indexes]
                image_sharp_batch = label[batch_indexes]

                generated_images = generator(image_blur_batch, training=True)

                real_output = discriminator(image_sharp_batch, training=True)
                fake_output = discriminator(generated_images, training=True)

                gen_loss = generator_loss(fake_output)
                disc_loss = discriminator_loss(real_output, fake_output)
              
            gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
            gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        display.clear_output(wait=True)

        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            save_all_weights(discriminator, generator, epoch, gen_loss)
        print(f'Time for epoch {epoch+1} is {time.time()-start} sec')
    display.clear_output(wait=True)

In [ ]:
BASE_DIR = '/content/drive/MyDrive/photo_results/'

def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join(BASE_DIR, '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)

In [ ]:
%%time
train(x_train, y_train, 300)

 18%|█▊        | 53/300 [1:54:44<8:53:34, 129.61s/it]

Time for epoch 53 is 129.69302225112915 sec


In [ ]:
Richter_path = '/content/drive/MyDrive/Colab Notebooks/Richter'

In [ ]:
RESHAPE = (256, 256)

def load_image(path):
    img = PIL.Image.open(path)
    return img

def preprocess_image(cv_img):
    cv_img = cv_img.resize(RESHAPE)
    img = np.array(cv_img)
    img = (img - 127.5) / 127.5
    return img

def deprocess_image(img):
    img = img * 127.5 + 127.5
    return img.astype('uint8')

def save_image(np_arr, path):
    img = np_arr * 127.5 + 127.5
    im = PIL.Image.fromarray(img)
    im.save(path)

In [ ]:
output_path = '/content/drive/MyDrive/photo_results/'
pretrained_path = '/content/drive/MyDrive/photo_results/725/generator_59_25.01026725769043.h5'

def deblur(weight_path, input_dir, output_dir):
    generator = generator_model()
    generator.load_weights(weight_path)
    for image_name in os.listdir(input_dir):
        image = np.array([preprocess_image(load_image(os.path.join(input_dir, image_name)))])
        x_test = image
        generated_images = generator.predict(x=x_test)
        generated = np.array([deprocess_image(img) for img in generated_images])
        x_test = deprocess_image(x_test)
        for i in range(generated_images.shape[0]):
            x = x_test[i, :, :, :]
            img = generated[i, :, :, :]
            output = np.concatenate((x, img), axis=1)
            im = PIL.Image.fromarray(output.astype(np.uint8))
            im.save(os.path.join(output_dir, image_name))

In [ ]:
deblur(pretrained_path, Richter_path, output_path)